# Convolutional Neural Network

### Importing the libraries

Computer Vision, Tensorflow, Pytorch, CNN, ... only work on Python

In [3]:
import tensorflow as tf
#from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
tf.__version__

'2.17.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

1. apply transformation only on training set to all images to avoid overfitting

2. This includes, shift, rotate, horizontal flips, zoom in and zoom out and we call it as image augmentation (augment the variety of image features to images)

3. Use Keras library for Deep Learning Computer Vision projects

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255, #feature scaling
                                   shear_range = 0.2, # these are image transformation
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [6]:
# don't apply any transformation on test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [7]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

C:\Users\sanaz\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [9]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
#pool_size = 2*2 squre
#strides: shifted two pixels from the left to the right (look at leture's note)

### Adding a second convolutional layer

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [11]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [12]:
# it should be fully connected --> Dense
# units = 128 hidden of fully connected layers
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [13]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [14]:
# adam optimizer for the gradient descent
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [17]:
import scipy.ndimage
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


C:\Users\sanaz\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 58s 224ms/step - accuracy: 0.5273 - loss: 0.6982 - val_accuracy: 0.5450 - val_loss: 0.6993
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 60ms/step - accuracy: 0.6187 - loss: 0.6484 - val_accuracy: 0.6965 - val_loss: 0.5872
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 61ms/step - accuracy: 0.6868 - loss: 0.5947 - val_accuracy: 0.7280 - val_loss: 0.5395
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 61ms/step - accuracy: 0.7119 - loss: 0.5548 - val_accuracy: 0.7440 - val_loss: 0.5323
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 61ms/step - accuracy: 0.7459 - loss: 0.5183 - val_accuracy: 0.7455 - val_loss: 0.5111
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 62ms/step - accuracy: 0.7544 - loss: 0.5055 - val_accuracy: 0.7580 - val_loss: 0.4980
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 64ms/step - accuracy: 0.7632 - loss: 0.4825 - val_accuracy: 0.7620 - val_loss: 0.5246
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 63ms/step - accuracy: 0.7738 - loss: 0.4657 - val_accura

## Part 4 - Making a single prediction

In [18]:
import numpy as np
from keras.preprocessing import image
# load image with the same target_size cat_or_dog_1 or cat_or_dog_2
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
# convert test_image to a numpy array
test_image = image.img_to_array(test_image)
# batch_size = 32 which is an extra dimension
# so to add extra (fake) dimension to the first dimension (index = 0):
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
# which indices corresponds to which image? 1 dog, 0 cat
training_set.class_indices
if result[0][0] == 1: #result[0][0] is our single element
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


In [19]:
print(prediction)

dog
